In [1]:
import os
import flamingo_model
from data_loaders import qa_dataset
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import sys
import optparse
import json
from tqdm import tqdm

In [2]:
data = qa_dataset.get_dataset(
            "MMQA", "val"
        )
data_loader = DataLoader(data)

In [3]:
from embedder.clip import CLIPEmbedder
import torch
device = "cuda:5" if torch.cuda.is_available() else "cpu"
# Define the model ID
model_ID = "openai/clip-vit-base-patch32"
clip = CLIPEmbedder(model_ID, device)

In [4]:
from vector_db.np_vector_db import NumpySearch
mmqa_retriever = NumpySearch(clip, "MMQA") 

loaded 275334 vectors
Loaded meta data from pickle file


In [5]:
model = flamingo_model.FlamingoModel("anas-awadalla/mpt-1b-redpajama-200b", "anas-awadalla/mpt-1b-redpajama-200b", 1)

Using pad_token, but it is not set yet.
/soe/sgarg6/.cache/huggingface/modules/transformers_modules/anas-awadalla/mpt-1b-redpajama-200b/8bc4eba452b5a5330f81975a761e4a59c851beea/attention.py:289: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn(


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.
Flamingo model initialized with 1046992944 trainable parameters


In [6]:
model.tokenizer

GPTNeoXTokenizerFast(name_or_path='anas-awadalla/mpt-1b-redpajama-200b', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<PAD>', 'additional_special_tokens': ['<|endofchunk|>', '<image>']}, clean_up_tokenization_spaces=True)

In [9]:
blank_image = Image.open("resources/1x1_#00000000.png")
answers = {}
ctr = 0
for x in tqdm(data_loader, position=0, leave=True):
    ques = x[0][0]
    print(ques)
    docs_retrieved = mmqa_retriever.retrieve(ques, "text")
    img_retrieved = mmqa_retriever.retrieve(ques, "image", k=5000)[:5]
    print(docs_retrieved)
    print(img_retrieved)
    qid = x[1][0]
    prompt = "<image>" + " <|endofchunk|> ".join([doc["text"] for doc in docs_retrieved[:2]]) + " <|endofchunk|> \n Answer the following question: \n " + ques + "\n Answer:" 
    print(prompt)
    ans = model.generate_answer(8, [blank_image], prompt)
    answers[qid] = ans
    ctr += 1
    if ctr == 5:
        break

  0%|                                                                                                                                     | 0/1176 [00:00<?, ?it/s]

Which Title(s), in Filmography of Ben Piazza, has the left half of a woman's face on its poster?


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


[{'title': 'Oh, What a Night (1944 film)', 'url': 'https://en.wikipedia.org/wiki/Oh,_What_a_Night_(1944_film)', 'id': '8a988bbbe5f583ddb32f4bb67905ebf1', 'text': "The film's story was written by Marian Orth and the screenplay by Paul Gerard Smith.", 'type': 'text'}, {'title': 'Reversion (2015 film)', 'url': 'https://en.wikipedia.org/wiki/Reversion_(2015_film)', 'id': '8e9145f2abf0d42710052056f8446932', 'text': 'The film has a 29% rating on Rotten Tomatoes. Wes Greene of "Slant Magazine" awarded the film two stars out of four.', 'type': 'text'}, {'title': '1979 in home video', 'url': 'https://en.wikipedia.org/wiki/1979_in_home_video', 'id': '76a62c2924a7089bda075913d59cb46e', 'text': 'The following movies were released on video on the following dates:', 'type': 'text'}, {'title': '1980 in home video', 'url': 'https://en.wikipedia.org/wiki/1980_in_home_video', 'id': 'd8100a1a139c8fc9820927d1bce03429', 'text': 'The following movies were released on video on the following dates:', 'type': 

  0%|                                                                                                                           | 1/1176 [00:11<3:37:35, 11.11s/it]

...

  0%|▍                                                                                                                          | 4/1176 [01:18<7:21:00, 22.58s/it]

What animals race in the Kentucky Derby?


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


[{'title': 'Kentucky Derby top four finishers', 'url': 'https://en.wikipedia.org/wiki/Kentucky_Derby_top_four_finishers', 'id': 'd22755592f600a3118c041cdf627b4b4', 'text': 'A † designates a Triple Crown Winner.', 'type': 'text'}, {'title': 'Belmont Stakes top three finishers', 'url': 'https://en.wikipedia.org/wiki/Belmont_Stakes_top_three_finishers', 'id': 'd32756fe297c4519591a147a5f453269', 'text': 'A † designates a Triple Crown Winner.', 'type': 'text'}, {'title': 'Eclipse Stakes (disambiguation)', 'url': 'https://en.wikipedia.org/wiki/Eclipse_Stakes_(disambiguation)', 'id': 'd8fd9ed03fab743d0dfa25d9d8bd2358', 'text': 'The Eclipse Stakes is a horse race in Great Britain.', 'type': 'text'}, {'title': '2012 Belmont Stakes', 'url': 'https://en.wikipedia.org/wiki/2012_Belmont_Stakes', 'id': '1c73de47d5d8623889d972b34b852c6f', 'text': "The race was run without a Triple Crown at stake as Kentucky Derby and Preakness Stakes winner I'll Have Another was scratched from the race due to a leg i

  0%|▍                                                                                                                          | 4/1176 [01:25<6:57:00, 21.35s/it]


In [10]:
answers

{'18ecd2ac6c0ac69993b92dc4b30137e8': 'The film\'s story was written by Marian Orth and the screenplay by Paul Gerard Smith. <|endofchunk|> The film has a 29% rating on Rotten Tomatoes. Wes Greene of "Slant Magazine" awarded the film two stars out of four. <|endofchunk|> \n Answer the following question: \n Which Title(s), in Filmography of Ben Piazza, has the left half of a woman\'s face on its poster?\n Answer: \n Which Title(s), in Filmography of Ben Piazza, has the right half of a woman\'s face on its poster?\n Answer: \n Which Title(s), in Filmography of',
 'e1e6ed53f9ad11813845088f4cf2f6b1': 'Miroslav Bjeloš (; born 29 October 1990) is a Serbian footballer, who plays as a midfielder for Napredak Kruševac. <|endofchunk|> Ivan Vukomanović (Serbian Cyrillic: Иван Вукомановић; born 19 June 1977) is a Serbian football manager. <|endofchunk|> \n Answer the following question: \n Which Club(s), in Career statistics | Club of Dženis Beganović, have a flower on the logo?\n Answer: \n Which

In [24]:
from data_loaders import data_utils
data = data_utils.load_jsonl_file("/data/users/sgarg6/capstone/multimodalqa/MMQA_dev.jsonl")

In [28]:
print([sample for sample in data if sample["qid"] == '18ecd2ac6c0ac69993b92dc4b30137e8'])

[{'qid': '18ecd2ac6c0ac69993b92dc4b30137e8', 'question': "Which Title(s), in Filmography of Ben Piazza, has the left half of a woman's face on its poster?", 'answers': [{'answer': 'Tell Me That You Love Me, Junie Moon', 'type': 'string', 'modality': 'image', 'text_instances': [], 'table_indices': [], 'image_instances': [{'doc_id': 'ddf8b52a8400deaf05940c5cad8169cd', 'doc_part': 'image'}]}], 'metadata': {'wiki_entities_in_question': [], 'wiki_entities_in_answers': ['Tell Me That You Love Me, Junie Moon'], 'type': 'ImageListQ', 'modalities': ['image'], 'image_doc_ids': ['c29dbf5e372abe403f66c4845bf924b5', '4e9411dae3cbab2b5223b9dabf825684', 'ddf8b52a8400deaf05940c5cad8169cd', 'c67ed9c5f5ba2a12fa95610d523fe28d', '4b6d3e469ff8395a0f8c1a13c0422937', 'cadbc8e1fb23d7fe15b88ea1141d2dd2', '1c0d30d5776bb8ee954667c5277f4bf7', 'ea2173ed76c57ccbe743e87c195f4a37', '8b5a40ca458a36fb64a1785297b66fd8', '9061cf6560fc6305fbd1575097c11aa4', '7869011075ff671da6fd10497b262609', '86cac3116bf69a5e5b2c20866d03

In [43]:
test = "Entrance to the Grand Canal, Venice is an oil on canvas painting by Paul Signac, executed in 1905, now in the Toledo Museum of Art, Ohio, USA. It shows the entrance to the Grand Canal in Venice, with the Dogana da Mar and Santa Maria della Salute in the background."
mmqa_retriever.retrieve(test, "image", k=10)

[{'title': 'Hidden Palms',
  'url': 'https://en.wikipedia.org/wiki/Hidden_Palms',
  'id': '9522e2239ab2561e48ac7f3f4c2599e3',
  'path': '9522e2239ab2561e48ac7f3f4c2599e3.jpg',
  'type': 'img'},
 {'title': 'Wolfenstein: Cyberpilot',
  'url': 'https://en.wikipedia.org/wiki/Wolfenstein:_Cyberpilot',
  'id': '398e8ebacb95002f14ed428eb6ecf27e',
  'path': '398e8ebacb95002f14ed428eb6ecf27e.jpg',
  'type': 'img'},
 {'title': 'Girlfriends (U.S. TV series)',
  'url': 'https://en.wikipedia.org/wiki/Girlfriends_(U.S._TV_series)',
  'id': '1699ec924ad24479122f9c5f04a85137',
  'path': '1699ec924ad24479122f9c5f04a85137.jpg',
  'type': 'img'},
 {'title': 'Journey Through the Past (film)',
  'url': 'https://en.wikipedia.org/wiki/Journey_Through_the_Past_(film)',
  'id': '3911f534150a4a8316072205b6f9cd05',
  'path': '3911f534150a4a8316072205b6f9cd05.jpg',
  'type': 'img'},
 {'title': 'MADtv',
  'url': 'https://en.wikipedia.org/wiki/MADtv',
  'id': '52c5807b6d519e31ef31660052dc652b',
  'path': '52c5807b6